<a href="https://colab.research.google.com/github/RemanMahameed/Few_shot_captioning/blob/main/Copy_of_Few_shot_captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from datasets import load_dataset
# load the data
ds = load_dataset("reach-vb/pokemon-blip-captions", split="train[:700]")

#split the data to train and test sets
ds = ds.train_test_split(test_size=0.2, seed=42)

train_ds = ds["train"]
test_dataset  = ds["test"]
# split the training data to train and val sets
#TODO: check if val is needed!
train_val = train_ds.train_test_split(test_size=0.2, seed=42)
train_dataset = train_val["train"]
val_dataset   = train_val["test"]
# print(ds)
# print(train_val)
from textwrap import wrap
import matplotlib.pyplot as plt
import numpy as np


# print some samples to check how are they visulaized:

# def plot_images(images, captions):
#     plt.figure(figsize=(20,20))
#     for i in range(len(images)):
#         ax=plt.subplot(1, len(images),i+1)
#         caption=captions[i]
#         caption="\n".join(wrap(caption,12))
#         plt.title(caption)
#         plt.imshow(images[i])
#         plt.axis("off")
# sample_images_to_visualize=[np.array(train_dataset[i]["image"]) for i in range(5)]
# sample_captions=[train_dataset[i]["text"] for i in range(5)]
# plot_images(sample_images_to_visualize, sample_captions)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

data/train-00000-of-00001-566cc9b19d7203(…):   0%|          | 0.00/99.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/833 [00:00<?, ? examples/s]













##









In [ ]:
from transformers import BlipProcessor

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

def preprocess(example):
    inputs = processor(images=example["image"], text=example["text"], padding="max_length", max_length=32, return_tensors="pt")
    example["pixel_values"] = inputs["pixel_values"][0]
    example["labels"] = inputs["input_ids"][0]
    return example

train_dataset = train_dataset.map(preprocess).remove_columns(["image","text"]).set_format(type="torch")
val_dataset   = val_dataset.map(preprocess).remove_columns(["image","text"]).set_format(type="torch")
test_dataset  = test_dataset.map(preprocess).remove_columns(["image","text"]).set_format(type="torch")
print(len(train_dataset), len(val_dataset), len(test_dataset))
print(train_dataset.column_names)

AttributeError: 'NoneType' object has no attribute 'map'

Trainer

In [ ]:
from transformers import BlipForConditionalGeneration, TrainingArguments, Trainer
import torch

model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

training_args = TrainingArguments(
    output_dir="./blip-pokemon",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=20,
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    remove_unused_columns=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [ ]:
trainer.train()




ValueError: Trainer: training requires a train_dataset.